# TASK 3: Evaluation von Input-Techniken

In dieser Studie werden verschiedene Eingabemethoden miteinander verglichen. Dazu zählt neben Maus und Touchpad auch eine gestenbasierte Eingabe. Außerdem wurde die Auswirkung von Latenz in Betracht gezogen. Dafür wurde reziproke Antippaufgabe verwendet. 

## Methodik

### Apperatus

Die Teilnehmer führten die Studie auf einem Macbook Air M1 2020 durch. Für die Eingabe wurden folgende Methoden verwendet:
- Maus (Logitech MX Anywhere 3)
- Maus (Logitech MX Anywhere 3) mit 150ms Latenz
- Touchpad (vom Macbook)
- gestenbasierte Eingabe durch eine Hand

Die gestenbasierte Eingabe erfolgte über die Handerkennung der Google MediaPipe. Dabei konnte der Mauszeiger über die Mitte der Hand (erstes Gelenk des Mittelfingers) gesteuert werden, wobei der Linksklick über die Berührung von Daumen und Zeigefinger erfolgte.

Bei der reziproken Antippaufgabe wurden 9 Kreise in einem UserInterface dargestellt. Dabei wurden für die Breite und den Abstand der Kreise alle Kombinationen aus 3 verschiedenen Werten verwendet. Diese wurden in einem Durchlauf automatisch durch das Programm geändert. Für jede dieser 9 Kombinationen mussten die Teilnehmer pro Gerät je 5 Wiederholungen durchlaufen. Damit wurden pro Teilnehmer 180 Datenpunkte erfasst.

### Prozedur

Die Teilnehmer wurden begrüßt und gebeten, die Einverständniserklärung sorgfälltig durchzulesen und diese zu unterschreiben. 

Daraufhin sollte ein demographischer Fragebogen ausgefüllt werden, der neben Alter und Geschlecht auch die berufliche Situation und die Computernutzung erfasste.

Danach wurde ihnen das Vorgehen geschildert und die verschiedensten Eingabemethoden genau beschrieben. Ihnen wurde gesagt, dass sie mit verschiedenen Eingabegeräten aus vielen Kreisen jeweils den bunten so schnell wie möglich aber präzise anklicken sollen. Daraufhin wurde die Anwendung von der Versuchsleitung geöffnet und die richtigen Parameter eingestellt. Dem Teilnehmer wurde gesagt, welches Eingabegerät er verwenden soll und nochmals kurz die Nutzung erklärt. Die Erfassung startete dann mit dem ersten Klick. Sobald keine Kreise mehr angezeigt wurden, war die Aufgabe für das Eingabegerät beendet. Dergleiche Vorgang wurde auch für die anderen Eingabemethoden wiederholt. Wir haben die Reihenfolge der Eingabemethoden für jeden Teilnehmer variiert, um mögliche Lerneffekte auszugleichen. 

### Teilnehmer

An unserer Studie nahmen ... Teilnehmer im Alter von ... bis ... Jahren teil (SD = ..., M = ...). ... davon identifizierten sich als weiblich, ... Personen als männlich. Alle Teilnehmer waren Studierende und gaben an, Computer regelmäßig zu verwenden...

## Ergebnisse

## Probleme

Ein Problem der Studie war es, die verschiedenen Distanzen und Breite der Ziele zu definieren, da wir kein erkennbares muster implementieren wollten. Dafür haben wir die Werte mit einem vorher gesetzten Seed zufällig generiert.

Ein weiteres Problem war das erzeugen einer realistischen Latenz. Da wir für die Versuchsdurchführung Pyglet verwendeten, konnte die Latenz nicht mit einfachen event-handlern simuliert werden, was ein flackern des Zeigers zur Folge hatte.